In [219]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
import datetime 
import time
import pytz
import os

In [220]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
last_cats = cat_history_df[last_mask]['name'].unique()
this_cats = cat_history_df[this_mask]['name'].unique()

diff = [x for x in this_cats if x not in last_cats]
print(diff) 

### Testing Foster List Data Collection

In [62]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

### Create old and New Files

In [166]:
df = getFosterInfo()

In [108]:
#make copy for edits
new_df = df.copy()

#changes to fields in existing rows
new_df.loc[3, 'cat_summary'] = 'test test test'
new_df.loc[7, 'cat_description'] = 'test test test'

#cat gets reserved
new_df.loc[9, 'cat_name'] = new_df.loc[9, 'cat_name'] + ' (reserved)'

#cat removed from foster list 
new_df.drop(12, inplace=True)

#cat added to list 
new_row = {
    'cat_id':['test-cat'],
    'cat_name':['test cat'],
    'cat_summary':['test'],
    'cat_timing':['test'],
    'cat_description':['test'],
    'cat_image':['test'],
    'cat_link':['test'],
    'import_at':[datetime.fromtimestamp(int(time.time()), tz=pytz.utc)]
}
new_row_df = pd.DataFrame(new_row)
new_df = pd.concat([new_df, new_row_df])


In [109]:
new_df.to_csv('../output/foster_cats_test_new.csv', index=False)

### Compare Files

In [73]:
old_df = pd.read_csv('../output/foster_cats_test_old.csv')
new_df = pd.read_csv('../output/foster_cats_test_new.csv')

In [160]:
# merge dfs and compare
compare_df = pd.merge(old_df, new_df, how='outer', on='cat_id', suffixes=['_old','_new'])
loop_cols = [t for t in old_df.columns if 'cat_id' not in t and 'import_at' not in t]

for col in loop_cols:
    col_compare_df = compare_df[[c for c in compare_df.columns if col in c]]
    compare_df[f'was_{col}_modified'] = (col_compare_df[f'{col}_old'] != col_compare_df[f'{col}_new'])

# discard rows where nothing was changed
compare_df['was_modified'] = compare_df[[col for col in compare_df.columns \
                                        if 'was_' in col]].any(axis=1)
compare_df = compare_df[compare_df['was_modified'] == True]

# distinguish adds, deletions, modifications
def determine_change(old_val, new_val):
    if pd.isnull(old_val):
        return 'row_added'
    elif pd.isnull(new_val):
        return 'row_deleted'
    else:
        return 'row_modified'

compare_df['change_type'] = compare_df.apply(lambda x: determine_change(x['cat_name_old'], x['cat_name_new']), 
                                             axis=1)

In [179]:
change_df = pd.read_csv('../output/foster_change_log.csv')

In [183]:
if change_df[change_df['change_type'] == 'row_added'].shape[0] > 0:
    print('new cats added!')

new cats added!


In [194]:
send_email(df, alert_message='change log test email', sender=my_username, password=my_password)

In [204]:
from foster_cats import *

In [221]:
# get last run + current run
old_fosters_df = pd.read_csv('../src/data/latest_fosters.csv')
new_fosters_df = getFosterInfo()

# split out html info from current run to avoid clutter in .csv
# we only need html for email alerts we might send 
new_fosters_write_df = new_fosters_df.drop(columns=['full_html'])

# save current run as history for next comparison
new_fosters_write_df.to_csv('../src/data/latest_fosters.csv', index=False)

# compare new and old runs to determine changes
changes_df = compare_snapshots(old_fosters_df, new_fosters_write_df)


if changes_df.shape[0] > 0:
    # logging.info('there have been changes since the last pipeline run')
    # logging.info('summary of changes: ')
    # logging.info(changes_df['change_type'].value_counts())
    
    # append changes to change history if it exists
    path = '../output/foster_change_log.csv'
    if os.path.exists(path):
        old_changes_df = pd.read_csv(path)
        changes_df = pd.concat([old_changes_df, changes_df])
    changes_df.to_csv(path, index=False)
    
    # if there were rows added, trigger alert

else:
    # logging.info('no changes since previous pipeline run')
    print('hi')

hi


In [245]:
recipients = pd.read_csv('../src/data/recipient_list.csv')

In [250]:
recipient_list = ", ".join(recipients['email'].to_list())

In [252]:
import smtplib
from email.mime.text import MIMEText
from configparser import ConfigParser

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')
recipients = pd.read_csv('../src/data/recipient_list.csv')
recipient_list = recipients['email'].to_list()

sender_email = my_username
sender_password = my_password

subject = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M')}: [TEST] New foster cat alert! {new_fosters_df.loc[0]['cat_summary']}"
body = f'''
    <html>
        <body>
        <h1> New cat alert! </h1>
            {new_fosters_df.loc[0]['full_html']}
        </body>
    </html>
    '''
html_message = MIMEText(body, 'html')
html_message['Subject'] = subject
html_message['From'] = sender_email
html_message['To'] = ", ".join(recipient_list)

server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(sender_email, sender_password)
server.sendmail(sender_email, recipient_list, html_message.as_string())
server.quit()

(221,
 b'2.0.0 closing connection s20-20020a37a914000000b00741a984943fsm2351290qke.40 - gsmtp')

In [231]:
new_fosters_df

0    Annie Oakley is a gentle and snuggly girl that...
Name: cat_description, dtype: object

In [197]:
from configparser import ConfigParser
import ssl

parser = ConfigParser()
_ = parser.read('../credentials.cfg')
my_username = parser.get('my_email','my_username')
my_password = parser.get('my_email','my_password')

# Define the HTML document
body = '''
    <html>
        <body>
            <h1>Daily S&P 500 prices report</h1>
            <p>Hello, welcome to your report!</p>
        </body>
    </html>
    '''

# Generate today's date to be included in the email Subject
date_str = str(datetime.datetime.now())

# Create a MIMEMultipart class, and set up the From, To, Subject fields
email_message = MIMEMultipart()
email_message['From'] = my_username
email_message['To'] = my_username
email_message['Subject'] = f'Report email - {date_str}'

# Attach the html doc defined earlier, as a MIMEText html content type to the MIME message
email_message.attach(MIMEText(html, "html"))
# Convert it as a string
email_string = email_message.as_string()

# Connect to the Gmail SMTP server and Send Email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(my_username, my_password)
    server.sendmail(my_username, my_password, email_string)

AttributeError: module 'datetime' has no attribute 'now'

# Appendix (Scratch)

In [ ]:
### first go to individual cat page and you can get the attributes via the below:

cafe_prospect = cat_soup.find('div', class_='comp-kjevammt2 YzqVVZ')


health_concerns = cat_soup.find('div', class_='comp-kjevamml YzqVVZ')


upcoming_appointments = cat_soup.find('div', class_='comp-kjevammp3 YzqVVZ')



## Create Change Log for Cafe Cats

In [3]:
import sys
sys.path.insert(1, '../src')
from cafe_cats import *
from foster_cats import *
import re
from utilities import *

import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import pytz
import os
import logging

In [4]:
url = 'https://catcafebk.com/our-cats/?'
jobtime = datetime.fromtimestamp(int(time.time()), tz=pytz.utc)

cat_df = get_latest_snapshot(url, jobtime)
print('...retrieved latest cat data')

#check if file exists and append
if os.path.exists('cat_history.csv'):
    print('...adding latest cat data to cat data history')
    cat_history_df = pd.read_csv('cat_history.csv')
    cat_history_df = pd.concat([cat_history_df, cat_df])
    cat_history_df.to_csv('cat_history.csv', index=False)

#otherwise create the file anew
else:
    cat_df.to_csv('cat_history.csv', index=False)

Getting summary...
...done
Getting cat filter data...
Getting cafe_cat cats... 
Getting kid_approved cats... 
Getting dog_approved cats... 
Getting companion_cat cats... 
Getting bonded_pair cats... 
Getting single_cat cats... 
...done
Getting individual cat info (descriptions, type)...
...checking page for 125 cats...
...checked 20 of 125 cat pages...
...checked 40 of 125 cat pages...
...checked 60 of 125 cat pages...
...checked 80 of 125 cat pages...
...checked 100 of 125 cat pages...
...checked 120 of 125 cat pages...
...done
...retrieved latest cat data


In [9]:
cat_df_old = cat_df.copy()
cat_df_new = get_latest_snapshot(url, jobtime = datetime.fromtimestamp(int(time.time()), tz=pytz.utc))

Getting summary...
...done
Getting cat filter data...
Getting cafe_cat cats... 
Getting kid_approved cats... 
Getting dog_approved cats... 
Getting companion_cat cats... 
Getting bonded_pair cats... 
Getting single_cat cats... 
...done
Getting individual cat info (descriptions, type)...
...checking page for 125 cats...
...checked 20 of 125 cat pages...
...checked 40 of 125 cat pages...
...checked 60 of 125 cat pages...
...checked 80 of 125 cat pages...
...checked 100 of 125 cat pages...
...checked 120 of 125 cat pages...
...done


In [ ]:
cat_df_old